## Change the coordinate system and the extent of the population density datasets to those of NLCD datasets

In [1]:
from osgeo import gdal

In [2]:
src_file = "D:\\788P\\PopDen_2005\\PopDen_2005_MD.tif"
src_raster = gdal.Open(src_file)
dest_raster = "D:\\788P\\PopDen_2005\\PopDen_2005_MD_TUM.tif"
gdal.Warp(dest_raster, src_raster, outputBounds = [84559.5157762685, 4181513.94539689, 523939.515776269, 4464743.94539689],\
          dstSRS='EPSG:26918', xRes= '30', yRes='30')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000197F2CDD450> >

In [3]:
src_file = "D:\\788P\\PopDen_2010\\PopDen_2010_MD.tif"
src_raster = gdal.Open(src_file)
dest_raster = "D:\\788P\\PopDen_2010\\PopDen_2010_MD_TUM.tif"
gdal.Warp(dest_raster, src_raster, outputBounds = [84559.5157762685, 4181513.94539689, 523939.515776269, 4464743.94539689],\
          dstSRS='EPSG:26918', xRes= '30', yRes='30')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000197F2CDD510> >

In [4]:
src_file = "D:\\788P\\PopDen_2015\\PopDen_2015_MD.tif"
src_raster = gdal.Open(src_file)
dest_raster = "D:\\788P\\PopDen_2015\\PopDen_2015_MD_TUM.tif"
gdal.Warp(dest_raster, src_raster, outputBounds = [84559.5157762685, 4181513.94539689, 523939.515776269, 4464743.94539689],\
          dstSRS='EPSG:26918', xRes= '30', yRes='30')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000197F2CDD330> >

## Vectorize the population density data

In [1]:
import rasterio
from rasterio.features import shapes
import geopandas as gp

In [2]:
mask = None
with rasterio.Env():
    with rasterio.open('D:\\788P\\PopDen_2005\\PopDen_2005_MD_TUM.tif') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

gpd_polygonized_pop  = gp.GeoDataFrame.from_features(geoms)
gpd_polygonized_pop = gpd_polygonized_pop.set_crs("EPSG:26918")
gpd_polygonized_pop = gpd_polygonized_pop[:-1] # remove the -9999 huge ploygon
gpd_polygonized_pop.to_file("D:\\788P\\PopDen_2005\\PopDen05.shp")

In [3]:
gpd_polygonized_pop

,geometry,raster_val
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901
1,"POLYGON ((117139.516 4406783.945, 117139.516 4...",6.186978
2,"POLYGON ((116419.516 4406813.945, 116419.516 4...",11.784114
3,"POLYGON ((119269.516 4406693.945, 119269.516 4...",9.279783
4,"POLYGON ((118549.516 4406723.945, 118549.516 4...",10.927161
...,...,...
36866,"POLYGON ((407779.516 4202663.945, 407779.516 4...",0.000000
36867,"POLYGON ((425299.516 4198793.945, 425299.516 4...",2.138899
36868,"POLYGON ((426049.516 4198793.945, 426049.516 4...",2.081310
36869,"POLYGON ((421669.516 4200653.945, 421669.516 4...",0.000000


## Vectorize the permanent forest loss data

In [4]:
with rasterio.Env():
    with rasterio.open('D:\\788P\\loss\\NLCD_loss_2006.tif') as src:
        image = src.read(1)
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

gpd_polygonized_loss  = gp.GeoDataFrame.from_features(geoms)
gpd_polygonized_loss = gpd_polygonized_loss.set_crs("EPSG:26918")
gpd_polygonized_loss = gpd_polygonized_loss[:-1] # remove the 0 huge ploygon
gpd_polygonized_loss.to_file("D:\\788P\\PopDen_2005\\NLCD06.shp")

In [5]:
gpd_polygonized_loss

,geometry,raster_val
0,"POLYGON ((149599.516 4404953.945, 149599.516 4...",1.0
1,"POLYGON ((142909.516 4402523.945, 142909.516 4...",1.0
2,"POLYGON ((134599.516 4402493.945, 134599.516 4...",1.0
3,"POLYGON ((141829.516 4402373.945, 141829.516 4...",1.0
4,"POLYGON ((141919.516 4402343.945, 141919.516 4...",1.0
...,...,...
7706,"POLYGON ((452599.516 4213523.945, 452599.516 4...",1.0
7707,"POLYGON ((452569.516 4213463.945, 452569.516 4...",1.0
7708,"POLYGON ((450739.516 4212263.945, 450739.516 4...",1.0
7709,"POLYGON ((452269.516 4208303.945, 452269.516 4...",1.0


## Spatially join the two vectorized data together, clean the joint result, and then save it

In [11]:
gpd_pop_loss = gp.sjoin(gpd_polygonized_pop, gpd_polygonized_loss, how='left')
gpd_pop_loss = gpd_pop_loss.rename(columns={'raster_val_left': 'PopDen', 'raster_val_right': 'ForestLoss'})
gpd_pop_loss['ForestLoss'] = gpd_pop_loss['ForestLoss'].fillna(0)
gpd_pop_loss = gpd_pop_loss.drop(columns=['index_right'])
gpd_pop_loss.to_file("D:\\788P\\PopDen_2005\\PopDen05NLCD06.shp")
gpd_pop_loss

,geometry,PopDen,ForestLoss
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0
1,"POLYGON ((117139.516 4406783.945, 117139.516 4...",6.186978,0.0
2,"POLYGON ((116419.516 4406813.945, 116419.516 4...",11.784114,0.0
3,"POLYGON ((119269.516 4406693.945, 119269.516 4...",9.279783,0.0
4,"POLYGON ((118549.516 4406723.945, 118549.516 4...",10.927161,0.0
...,...,...,...
36866,"POLYGON ((407779.516 4202663.945, 407779.516 4...",0.000000,0.0
36867,"POLYGON ((425299.516 4198793.945, 425299.516 4...",2.138899,0.0
36868,"POLYGON ((426049.516 4198793.945, 426049.516 4...",2.081310,0.0
36869,"POLYGON ((421669.516 4200653.945, 421669.516 4...",0.000000,0.0


## Vectorize the NLCD landcover data

In [12]:
mask = None
with rasterio.Env():
    with rasterio.open('D:\\788P\\NLCD2004\\NLCD2004_MD_UTM.tif') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

gpd_polygonized_nlcd  = gp.GeoDataFrame.from_features(geoms)
gpd_polygonized_nlcd = gpd_polygonized_nlcd.set_crs("EPSG:26918")
gpd_polygonized_nlcd = gpd_polygonized_nlcd[:-1] # remove the -9999 huge ploygon
#gpd_polygonized_nlcd.to_file("D:\\788P\\NLCD2004\\NLCD04.shp")

In [13]:
gpd_polygonized_nlcd

,geometry,raster_val
0,"POLYGON ((117439.516 4406363.945, 117439.516 4...",41.0
1,"POLYGON ((117439.516 4406333.945, 117439.516 4...",43.0
2,"POLYGON ((117769.516 4406333.945, 117769.516 4...",41.0
3,"POLYGON ((117949.516 4406333.945, 117949.516 4...",42.0
4,"POLYGON ((117379.516 4406303.945, 117379.516 4...",43.0
...,...,...
1701626,"POLYGON ((420859.516 4194743.945, 420859.516 4...",255.0
1701627,"POLYGON ((420889.516 4194713.945, 420889.516 4...",255.0
1701628,"POLYGON ((420799.516 4194953.945, 420799.516 4...",95.0
1701629,"POLYGON ((420949.516 4194623.945, 420949.516 4...",95.0


## Spatially join the vectorized NLCD data to the previous joint result, clean the new joint result, and save it

In [14]:
gpd_pop_loss_nlcd = gp.sjoin(gpd_pop_loss, gpd_polygonized_nlcd, how='left')
gpd_pop_loss_nlcd = gpd_pop_loss_nlcd.drop(columns=['index_right'])
gpd_pop_loss_nlcd = gpd_pop_loss_nlcd.rename(columns={'raster_val': 'LandCover'})
gpd_pop_loss_nlcd

,geometry,PopDen,ForestLoss,LandCover
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0,42.0
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0,42.0
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0,43.0
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0,90.0
0,"POLYGON ((117829.516 4406753.945, 117829.516 4...",7.075901,0.0,43.0
...,...,...,...,...
36870,"POLYGON ((420889.516 4196033.945, 420889.516 4...",0.000000,0.0,11.0
36870,"POLYGON ((420889.516 4196033.945, 420889.516 4...",0.000000,0.0,11.0
36870,"POLYGON ((420889.516 4196033.945, 420889.516 4...",0.000000,0.0,95.0
36870,"POLYGON ((420889.516 4196033.945, 420889.516 4...",0.000000,0.0,11.0


In [18]:
gpd_pop_loss_nlcd.to_file("D:\\788P\\NLCD2004\\PopDen05NLCD06LC.shp")